In [4]:
import cv2
import numpy as np
import os
import tkinter as tk
import re
from tkinter import filedialog, messagebox
from plotly import graph_objects as go
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Function to extract features from an image using SIFT
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

# Function to match features between two images using Brute-Force matcher
def match_features(descriptors1, descriptors2):
    matcher = cv2.BFMatcher(cv2.NORM_L2)
    if descriptors1.dtype != descriptors2.dtype:
        descriptors2 = descriptors2.astype(descriptors1.dtype)
    matches = matcher.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

# Function to estimate the alignment parameters (translation) from matched keypoints
def estimate_alignment(matches, keypoints1, keypoints2):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    dx = M[0, 2]
    dy = M[1, 2]
    return dx, dy

# Function to train a regression model on alignment parameters
def train_model(features, alignment_parameters, model=None):
    if model is None:
        model = tf.keras.Sequential([
            layers.Dense(32, activation='relu', input_shape=(len(features[0]),)),
            layers.Dense(32, activation='relu'),
            layers.Dense(2)  # Two outputs: dx and dy
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')

    X_train, X_val, y_train, y_val = train_test_split(features, alignment_parameters, test_size=0.2, random_state=42)

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)
    return model, history

# Function to select the folder using Tkinter
def select_folder():
    root = tk.Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory(title='Select Input Folder')
    root.destroy()  # Destroy the root window
    return folder_selected

def select_model_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title='Select Model File', filetypes=[('Model Files', '*.h5')])
    root.destroy()  # Destroy the root window
    return file_path

def select_output_folder():
    root = tk.Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory(title='Select Output Folder for Training Report')
    root.destroy()  # Destroy the root window
    return folder_selected

# Function to display the training and validation plot
def display_training_plot(history):
    plot_metrics(history)

# Function to log the training metrics to a file
def log_metrics(metrics, file_path):
    with open(file_path, 'w') as f:
        f.write('Training Metrics:\n')
        for metric_name, metric_values in metrics.items():
            f.write(f'{metric_name}: {metric_values}\n')

# Function to plot the training metrics
def plot_metrics(history):
    loss = history.history['loss']
    val_loss = history.history.get('val_loss', [])
    epochs = list(range(1, len(loss) + 1))  # Convert epochs to a list

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs, y=loss, name='Training Loss'))
    fig.add_trace(go.Scatter(x=epochs, y=val_loss, name='Validation Loss'))

    fig.update_layout(
        title='Training Metrics',
        xaxis_title='Epochs',
        yaxis_title='Loss',
        xaxis=dict(title='Epochs'),
        yaxis=dict(title='Loss'),
        xaxis_tickmode='linear',
        xaxis_dtick=1  # Increase the granularity of x-axis ticks
    )

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')  # Add axis line
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')

    fig.show()

# Function to continue training on an existing model
def continue_training_model(features, alignment_parameters, model):
    # Adjust the input shape of the model if needed
    if model.input_shape[1] != features.shape[1]:
        model = adjust_model_input_shape(model, features.shape[1])

    history = model.fit(features, alignment_parameters, epochs=10, batch_size=16)
    return model, history

def adjust_model_input_shape(model, new_input_shape):
    # Create a new model with adjusted input shape
    new_model = tf.keras.Sequential()
    for layer in model.layers:
        if isinstance(layer, layers.Dense):
            new_layer = layers.Dense(layer.units, activation=layer.activation, input_shape=(new_input_shape,))
        else:
            new_layer = layer
        new_model.add(new_layer)
    new_model.compile(optimizer=model.optimizer, loss=model.loss)
    return new_model

# Function to run alignment using an existing model
def run_alignment_model(image_files, input_folder, ideal_image, model, training_report_folder):
    keypoints_ideal, descriptors_ideal = extract_features(ideal_image)

    if descriptors_ideal is None:
        failed_images_feature_extraction = [image_files[0]]  # Add the ideal image to the list of failed feature extraction
    else:
        failed_images_feature_extraction = []  # Initialize the list of images with failed feature extraction

    # Perform alignment for each image
    alignment_distances = []  # Initialize the alignment distances list
    for image_file in image_files:
        image = cv2.imread(os.path.join(input_folder, image_file))
        keypoints, descriptors = extract_features(image)

        if descriptors is None:
            failed_images_feature_extraction.append(image_file)  # Add the image filename to the list of failed feature extraction
            continue

        matches = match_features(descriptors_ideal, descriptors)

        if matches is None:
            failed_images_feature_extraction.append(image_file)  # Add the image filename to the list of failed feature extraction
            continue

        dx, dy = estimate_alignment(matches, keypoints_ideal, keypoints)
        distance = np.sqrt(dx ** 2 + dy ** 2)
        alignment_distances.append((image_file, dx, dy, distance))

        # Draw aligned images and save them
        aligned_image_file = os.path.join(training_report_folder, f'aligned_image_{image_file}')
        feature_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 0, 255), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imwrite(aligned_image_file, feature_image)

    # Sort the alignment distances numerically based on the image file name
    alignment_distances.sort(key=lambda x: int(re.search(r'\d+', x[0]).group()) if re.search(r'\d+', x[0]) else float('inf'))

    # Save alignment distances to the ideal image in the training report
    with open(os.path.join(training_report_folder, 'alignment_distances.txt'), 'w') as f:
        f.write('Alignment Distances to Ideal Image:\n')
        for image_file, dx, dy, distance in alignment_distances:
            f.write(f'{image_file}: x: {dx}, y: {dy}, total: {distance}\n')

    # Update the training report with the failed images for feature extraction
    with open(os.path.join(training_report_folder, 'training_report.txt'), 'w') as f:
        num_failed_images = len(failed_images_feature_extraction)
        num_accurately_aligned_images = len(alignment_distances) - num_failed_images
        f.write(f'Number of Accurately Aligned Images: {num_accurately_aligned_images}\n')
        f.write('Images with Failed Feature Extraction:\n')
        for image_file in failed_images_feature_extraction:
            f.write(f'{image_file}\n')

    # Calculate alignment success rate
    total_images = len(image_files) - 1  # Excluding the ideal image
    alignment_success_rate = (num_accurately_aligned_images / total_images) * 100

    # Update the training report with the alignment success rate
    with open(os.path.join(training_report_folder, 'training_report.txt'), 'a') as f:
        f.write(f'Alignment Success Rate: {alignment_success_rate:.2f}%\n')

    messagebox.showinfo('Training Complete', 'Evaluation complete! Check the training report for details.')

# Main script
root = tk.Tk()
root.withdraw()

input_folder = select_folder()
if input_folder is not None:
    features = []
    alignment_parameters = []
    failed_feature_extraction_images = []  # List to store images with failed feature extraction

    # Find the ideal image with '_ideal' in the filename
    image_files = sorted([filename for filename in os.listdir(input_folder) if filename.endswith('.png')])
    for filename in image_files:
        if '_ideal' in filename:
            ideal_image = cv2.imread(os.path.join(input_folder, filename))
            break

    # Read training images and extract features and alignment parameters
    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        keypoints_auto, descriptors_auto = extract_features(image)

        if descriptors_auto is not None:
            features.append(descriptors_auto.flatten())
            dx, dy = get_alignment_parameters()  # Function to get alignment parameters
            alignment_parameters.append([dx, dy])
        else:
            failed_feature_extraction_images.append(filename)  # Store the image with failed feature extraction

    # Check if any training images were successfully processed
    if len(features) == 0:
        messagebox.showerror('No Training Images', 'No training images found or feature extraction failed for all images. Cannot proceed with training.')
        sys.exit(1)

    # Determine the maximum length of feature vectors
    max_length = max(len(feature) for feature in features)

    # Pad shorter feature vectors with zeros
    padded_features = []
    for feature in features:
        padded_feature = np.pad(feature, (0, max_length - len(feature)), mode='constant')
        padded_features.append(padded_feature)

    # Convert lists to NumPy arrays
    features = np.array(padded_features)
    alignment_parameters = np.array(alignment_parameters)

    # Select the output folder for saving the training report
    training_report_folder = select_output_folder()

    # Main script
    root = tk.Tk()
    root.withdraw()

    choice = messagebox.askquestion('Model Selection', 'Do you want to load an existing model?')

    if choice == 'yes':
        model_file = select_model_file()
        model = tf.keras.models.load_model(model_file)

        continue_training_choice = messagebox.askquestion('Model Usage', 'Do you want to continue training on the old model?')

        if continue_training_choice == 'yes':
            # Continue training on the existing model
            model, history = continue_training_model(features, alignment_parameters, model)
            display_training_plot(history)  # Display the training and validation plot
            save_choice = messagebox.askquestion('Save Model', 'Do you want to save the trained model?')
            if save_choice == 'yes':
                model_path = filedialog.asksaveasfilename(title='Save Trained Model',
                                                          defaultextension='.h5',
                                                          filetypes=[('Model Files', '*.h5')])
                model.save(model_path)
        else:
            # Run alignment using the existing model
            run_alignment_model(image_files, input_folder, ideal_image, model, training_report_folder)
    else:
        # Create a new model and perform training
        model = None

        # Train the regression model
        model, history = train_model(features, alignment_parameters, model)

        # Save the trained model
        save_choice = messagebox.askquestion('Save Model', 'Do you want to save the trained model?')
        if save_choice == 'yes':
            model_path = filedialog.asksaveasfilename(title='Save Trained Model',
                                                      defaultextension='.h5',
                                                      filetypes=[('Model Files', '*.h5')])
            model.save(model_path)

        # Log and save the training metrics
        training_metrics = {
            'Loss': history.history['loss'],
            'Validation Loss': history.history.get('val_loss', [])
        }
        log_metrics(training_metrics, os.path.join(training_report_folder, 'training_metrics.txt'))

        # Update the alignment process if an ideal image is selected
        if ideal_image is not None:
            run_alignment_model(image_files, input_folder, ideal_image, model, training_report_folder)

    messagebox.showinfo('Training Complete', 'Training and evaluation complete! Check the training report for details.')
    root.destroy()


Epoch 1/10
8/8 [==============================] - 1s 60ms/step - loss: 1024.6816 - val_loss: 273.7853
Epoch 2/10
8/8 [==============================] - 0s 10ms/step - loss: 278.5581 - val_loss: 208.6237
Epoch 3/10
8/8 [==============================] - 0s 12ms/step - loss: 181.5950 - val_loss: 125.7610
Epoch 4/10
8/8 [==============================] - 0s 23ms/step - loss: 162.1156 - val_loss: 74.8452
Epoch 5/10
8/8 [==============================] - 0s 26ms/step - loss: 105.2366 - val_loss: 74.4359
Epoch 6/10
8/8 [==============================] - 0s 12ms/step - loss: 72.5958 - val_loss: 126.6922
Epoch 7/10
8/8 [==============================] - 0s 32ms/step - loss: 53.3616 - val_loss: 50.4831
Epoch 8/10
8/8 [==============================] - 0s 16ms/step - loss: 35.6386 - val_loss: 47.0732
Epoch 9/10
8/8 [==============================] - 0s 20ms/step - loss: 22.6804 - val_loss: 44.4999
Epoch 10/10
8/8 [==============================] - 0s 14ms/step - loss: 11.0238 - val_loss: 35.9750